**<font size=5>Tennessee Fuel Quality Analysis</font>**

* **Date Published**: 2019/07/31
* **Collaborators**: Kate Hayes & Misha Berrien 
* **Data Source**: ? 

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import seaborn as sns
import sklearn.preprocessing as preprocessing
import statsmodels.api as sm
import sys

from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, ADASYN


src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

In [5]:
# helper functions 
from d03_processing.feature_engineering import process_data_for_model_building
from d04_modelling.modelling import get_model_pvalue
from d03_processing.Time_series_cleaning import date_results_df_creator
from d03_processing.Time_series_cleaning import volatilty_ASTM_df_creator


# Load the "autoreload" extension
%load_ext autoreload

# reload modules so that as you change code in src, it gets loaded
%autoreload

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Introduction 

## Dataset

The state of Tennessee's Department of Agriculture (TDA) maintaines a fuel quality program. The state inspects all places where fuel is sold/ distributed including gas stations, terminals, airports etc. on an annual basis. This results of these routine tests as well as follow up tests for complaints are available. Tennessee has a Sunshine law (public records law) that allows anyone to request any state record through the right legal channels. The data set was aquired in this manor. We sent an official Tennessee records request for the fuel qulity data for the last 5 years. It came in the form of routine inspections and complaints one set for each years, so 10 Excel files in total.

## Key Metrics

#### The following key metrics were descriptions given to us by the TDA Fuel Quality Manager. The main thing that we were inspecting in this project was the prediction of pass/fail rates and volatility properties with the seasons. 

Volatility is an important property of gasoline because it must be able to vaporize before combusting in an engine. Three characteristics are used to measure the volatility of gasoline and evaluate suitability: vapor pressure, vapor-liquid ratio, and the distillation temperature at which 50% of the fuel is evaporated.


* Vapor pressure is a measure of the amount of vapor that is produced by a gasoline sample at 37.8°C (100°F). Vapor pressure most affects an engine’s ease of starting. The vapor pressure specification is a maximum allowable limit reported in kilopascals. The pressure must be high enough to promote easy starting but not too high to contribute to excessive emissions or vapor lock -  the presence of too much vapor that leads to loss of engine power or rough operation.


* Vapor-liquid ratio is the ratio of the volume of vapor to the volume of liquid at atmospheric pressure. The vapor-liquid ratio specification is a minimum allowable limit reported in degrees Celsius. The reported value is the temperature at which the vapor-liquid ratio is equal to 20 (20:1 vol/vol), the approximate temperature at which engine problems may occur. Vapor-liquid ratio is used to evaluate a gasoline sample’s tolerance to changes in temperature. A noncompliant test result (too low) may lead to vapor lock or hot fuel handling problems, as evidenced by loss of power while accelerating or idling.


* Distillation measures the temperature range across which a sample is heated to fully evaporate. The temperature at which 50% of a sample is evaporated (T50) relates to the driveability (smoothness and ease of driving) and idling characteristics for the fuel. T50 most similarly relates to how a fuel performs under continuous activity (not starting or warming up). T50 has minimum and maximum allowable limits reported in degrees Celsius.

## Question

## Caveats/ shortcomings/ issues (whatever you would name this section) 

## Time Series Analysis

### Load and Process Datasets 

The fuel volatility specifications change throughout the year based on outdoor temperature. Therse specifications are set by the ASTM. The ASTM standards were joined onto the five year result data frame. in order to assess the seasonality of the data. A Dicky Fuller test and selective seaonal decomposition was done.

In [10]:
volatility_and_astm_df = volatilty_ASTM_df_creator('../../data/02_intermediate/routine_clean.csv', '../../data/01_raw/ASTM_fuel.csv')
volatility_and_astm_df.head()

               Sample      Prod DateSampled               Grade  \
DateSampled                                                       
2015-11-23   61916134  Gasoline  2015-11-23  Mid Grade Unleaded   
2015-11-23   61916134  Gasoline  2015-11-23  Mid Grade Unleaded   
2015-11-23   61916134  Gasoline  2015-11-23  Mid Grade Unleaded   
2015-11-23   61916134  Gasoline  2015-11-23  Mid Grade Unleaded   
2015-11-23   61916134  Gasoline  2015-11-23  Mid Grade Unleaded   

                          Supplier       FacilityName  \
DateSampled                                             
2015-11-23   Marathon Petroleum Lp  Circle K #2723609   
2015-11-23   Marathon Petroleum Lp  Circle K #2723609   
2015-11-23   Marathon Petroleum Lp  Circle K #2723609   
2015-11-23   Marathon Petroleum Lp  Circle K #2723609   
2015-11-23   Marathon Petroleum Lp  Circle K #2723609   

                                        SiteAddress              Test   Units  \
DateSampled                                      

NameError: name 'full_volatility_df' is not defined

In [ ]:
distillaltion_df = date_results_df_creator(volatility_and_astm_df, 'Distillation 50%')

In [ ]:
vapor_pressure_df = date_results_df_creator(volatility_and_astm_df, 'Vapor Pressure')

In [ ]:
vapor_liquid_df = date_results_df_creator(volatility_and_astm_df, 'Vapor-Liquid Ratio')

### Dicky-Fuller Stationality Tests
Once the proper dataframes were created a Dicky Fuller Test was added to assess for stationarity. All of the tests were stationary above a 1% critical value. Two failed at 1% but that was due to extreme sample failures. The average, maximum, and minimum values for each day were taken becasue there was more than one value for each date and time series analysis requires there to be no more than one value for each date. Assessing at these points allowed differing seasonal trends to be seen and the outliers on either side of the spectrum to become more obvious. It is not the most informative thing to say that the value of one sample in Knoxville averaged with another in Nashville can tell the user anything about the trends. It can disguies a lot of the analysis

In [ ]:
stationarity_check(Distillation_50.groupby('DateSampled').mean(), 'Distillation 50 % Mean')

In [ ]:
stationarity_check(Vapor_Liquid.groupby('DateSampled').mean(), 'Vapor Liquid Mean')

In [ ]:
stationarity_check(Vapor_Pressure.groupby('DateSampled').mean(), 'Vapor Pressure Mean')

In [ ]:
stationarity_check(Distillation_50.groupby('DateSampled').max(), 'Distillation 50 % Mean')

In [ ]:
stationarity_check(Vapor_Liquid.groupby('DateSampled').max(), 'Vapor Liquid Mean')

In [ ]:
stationarity_check(Vapor_Pressure.groupby('DateSampled').mean(), 'Vapor Pressure Mean')

In [ ]:
stationarity_check(Distillation_50.groupby('DateSampled').min(), 'Distillation 50 % Mean')

In [ ]:
stationarity_check(Vapor_Liquid.groupby('DateSampled').min(), 'Vapor Liquid Mean')

In [ ]:
stationarity_check(Vapor_Pressure.groupby('DateSampled').min(), 'Vapor Pressure Mean')

### Time Series Decomposition
A time series decomposition on the average vapor pressure was done. This was selected to do the decomposition becasue it had an easy to see trend of seasonality with few outliers.

In [ ]:
# Make A resampled vapor pressure data frame that has frequency by buisness days and the missing dates are upsampled


In [ ]:
seasonal_decompose(resampled_vp, 'Average Vapor Pressue Seasonal Decomposition')

The frequency was done via buisness days so becasue of that it does not pull out the quarterly trend data.

## Logistic Regression Analysis

### Load and Process Datasets

The helper functions for cleaning/ processing the dataset can be found in the src/d03_processing folder

In [ ]:
gasoline_proc = pd.read_csv('../../data/03_processed/gasoline_processed.csv')
astm = pd.read_csv('../../data/01_raw/ASTM_fuel.csv')
astm.columns = ['Date', 'TN_retailers_seasons', 'TN_distributor_seasons',
       'vapor_liquid_minC_retail', 'distillation_50_minC _retail',
       'distillation_50_maxC_retail', 'vapor_pressure_maxC_retail',
       'vapor_liquid_minC_dist', 'distillation_50_minC_dist',
       'distillation_50_maxC_dist', 'vapor_pressure_maxC_dist']

In [ ]:
gasoline = process_data_for_model_building(gasoline_proc, astm)

### Build & Choose Model

#### Define Variables

In [ ]:
# construct features 
x_feats = ['grade']
X = pd.get_dummies(gasoline[x_feats], dtype=float)
X = sm.tools.add_constant(X)
# convert target using get_dummies
y = pd.get_dummies(gasoline["compliance_vap_liq_pressure"], dtype=float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y.iloc[:,0], test_size=0.3, random_state=0)

Our dataset is heavily imbalanced with a 26:9062 ratio of 1 to 0s. In order to find a reliable result, we need to balance these numbers with oversampling. 

In [ ]:
print("Label Count '1': {}".format(sum(y_train==1)))
print("Label Count '0': {} \n".format(sum(y_train==0)))

#### Oversampling

In [ ]:
smote = SMOTE()

# simple resampling from your previously split data
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train.ravel())

In [ ]:
print("Label Count '1': {}".format(sum(y_train_resampled==1)))
print("Label Count '0': {} \n".format(sum(y_train_resampled==0)))

We now have a balanced dataset to build our models with. 

#### Model 1: Vapor Liquid-Ratio Test Outcome ~ Grade

In [ ]:
logreg = LogisticRegression(fit_intercept = False, C = 1e12)
model_log = logreg.fit(X_train_resampled, y_train_resampled)
model_log

Let's find our pvalues

In [ ]:
get_model_pvalue(y_train_resampled, X_train_resampled)

These pvalues are not significant. Let's try another model. 

#### Model 2: Vapor Liquid-Ratio Test Outcome ~ Tennessee Retailers Season & Grade

We will repeat the process laid out above for our second model

In [ ]:
# construct features 
x_feats = ['TN_retailers_seasons', 'grade']
X = pd.get_dummies(gasoline[x_feats], dtype=float)
X = sm.tools.add_constant(X)
# convert target using get_dummies
y = pd.get_dummies(gasoline["compliance_vap_liq_pressure"], dtype=float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y.iloc[:,0], test_size=0.3, random_state=0)

In [ ]:
smote = SMOTE()

# simple resampling from your previously split data
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train.ravel())

In [ ]:
# simple resampling from your previously split data
X_test_resampled, y_test_resampled = smote.fit_sample(X_test, y_test.ravel())

In [ ]:
logreg = LogisticRegression(fit_intercept = False, C = 1e12)
model_log = logreg.fit(X_train_resampled, y_train_resampled)
model_log

In [ ]:
get_model_pvalue(y_train_resampled, X_train_resampled)

Neither of our models have significant pvalues, but, since we want to understand a bit better about blah blah blah blah, we are going to choose the second model (which have slightly higher pvalues) and move on to our testing phase. 

### Test Chosen Model (Prediction) 

In [ ]:
y_hat_test_resampled = logreg.predict(X_test_resampled)
y_hat_train_resampled = logreg.predict(X_train_resampled)

#### Precision, Recall, Accuracy and F1-Score

In [ ]:
print('Training Precision: ', precision_score(y_hat_train_resampled, y_train_resampled))
print('Testing Precision: ', precision_score(y_hat_test_resampled, y_test_resampled))
print('\n\n')

print('Training Recall: ', recall_score(y_hat_train_resampled, y_train_resampled))
print('Testing Recall: ', recall_score(y_hat_test_resampled, y_test_resampled))
print('\n\n')

print('Training Accuracy: ', accuracy_score(y_hat_train_resampled, y_train_resampled))
print('Testing Accuracy: ', accuracy_score(y_hat_test_resampled, y_test_resampled))
print('\n\n')

print('Training F1-Score: ',f1_score(y_hat_train_resampled, y_train_resampled))
print('Testing F1-Score: ',f1_score(y_hat_test_resampled, y_test_resampled))

#### ROC Curve & AUC

In [ ]:
#First calculate the probability scores of each of the datapoints:
y_score = logreg.fit(X_train_resampled, y_train_resampled).decision_function(X_test_resampled)

fpr, tpr, thresholds = roc_curve(y_test_resampled, y_score)

In [ ]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})

print('AUC: {}'.format(auc(fpr, tpr)))
plt.figure(figsize=(10,8))
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve')
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.yticks([i/20.0 for i in range(21)])
plt.xticks([i/20.0 for i in range(21)])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

### Logistic Regression Conclusion

BLAH BLAH BLAH 

## Conclusion